![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [104]:
import pandas as pd
import numpy as np

# Start coding here...

In [105]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


# **Step 1: Creating 3 DataFrames**

The first step involves splitting our data into client, campaign, and ecconomies dataframes. We can do this by taking specific variable listed above.

In [106]:
# Split into the 3 DataFrames
client = df[["client_id", "age", "job", "marital", 
                    "education", "credit_default", "mortgage"]]
campaign = df[["client_id", "number_contacts", "month", "day", 
               "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]]
economics = df[["client_id", "cons_price_idx", "euribor_three_months"]]

print(client)
print(campaign)
print(economics)

       client_id  age          job  ...            education credit_default mortgage
0              0   56    housemaid  ...             basic.4y             no       no
1              1   57     services  ...          high.school        unknown       no
2              2   37     services  ...          high.school             no      yes
3              3   40       admin.  ...             basic.6y             no       no
4              4   56     services  ...          high.school             no       no
...          ...  ...          ...  ...                  ...            ...      ...
41183      41183   73      retired  ...  professional.course             no      yes
41184      41184   46  blue-collar  ...  professional.course             no       no
41185      41185   56      retired  ...    university.degree             no      yes
41186      41186   44   technician  ...  professional.course             no       no
41187      41187   74      retired  ...  professional.course     

# **Step 2: Cleaning Our Data**

Now its time to clean. Given the requirements above, we have a few guidelines to follow. We can start with our client dataframe

## Cleaning Client DataFrame

In [107]:
# Change job variable "." to "_"
client['job'] = client['job'].str.replace('.', '_')

# Change education variable "." to "_" and "unknown" to "np.NAN"
client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.NaN)

# Convert credit_default and mortgage to boolean data type
for column in ['credit_default', 'mortgage']:
    client[column] = client[column].map({
        "yes": 1,
        "no": 0,
        "unknown": 0
    })
    client[column] = client[column].astype(bool)

# Checking to see if it is cleaned
client.head()
#client.info()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False


We have checked to see that the client dataframe is cleaned and has the correct variable types. Now we can move on to cleaning our second dataframe, campaign.

## Cleaning Campaign DataFrame

In [108]:
# Convering previous_outcome and campaign_outcome into boolean
campaign["previous_outcome"] = campaign["previous_outcome"].map({
        "success": 1,
        "nonexistent": 0,
        "failure": 0,
    })
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({
    "yes": 1,
    "no": 0
})

for column in ["campaign_outcome", "previous_outcome"]:
  campaign[column] = campaign[column].astype(bool)

In [109]:
#Adding the year column as stated
campaign['year'] = "2022"

#changing the "day" dtype from int to str
campaign["day"] = campaign["day"].astype("str")

#combining the day, month and year
campaign["last_contact_date"] = campaign["year"]+ "-" + campaign["month"]+ "-" + campaign["day"]

#changing the format
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"])

#dropping extra columns
campaign.drop(columns=['year', 'day', 'month'], inplace=True)

#checking the datatype
print(campaign.dtypes)
print(campaign.head())

client_id                              int64
number_contacts                        int64
contact_duration                       int64
previous_campaign_contacts             int64
previous_outcome                        bool
campaign_outcome                        bool
last_contact_date             datetime64[ns]
dtype: object
   client_id  number_contacts  ...  campaign_outcome  last_contact_date
0          0                1  ...             False         2022-05-13
1          1                1  ...             False         2022-05-19
2          2                1  ...             False         2022-05-23
3          3                1  ...             False         2022-05-27
4          4                1  ...             False         2022-05-03

[5 rows x 7 columns]


Now, we have finished cleaning and checking our campaign dataframes. We can finally work on exporting the dataframes as CSV files

# **Step 3: Exporting Our Data**

In [111]:
# Save DataFrames to CSV files without an index
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)